In [314]:
import os
import datetime
import time
import requests
import pandas as pd
import json
import ast

from geopy.geocoders import Nominatim

In [126]:
with open('target_cities.json') as json_file:
    target_cities = json.load(json_file)

# Representing the Target cities 

In [133]:
for i in target_cities:
    print(i)
    print(len(target_cities[i]))

EU
17
US
13
Seattle
15


In [134]:
from pprint import pprint

pprint(target_cities)

{'EU': ['Amsterdam',
        'Athina',
        'Berlin',
        'Gdansk',
        'Kraków',
        'London',
        'Madrid',
        'Marseille',
        'Milano',
        'München',
        'Napoli',
        'Paris',
        'Sevilla',
        'Stockholm',
        'Tallinn',
        'Varna',
        'Wien'],
 'Seattle': {'Seattle - Bellevue-SE 12th St': (47.6008630009392, -122.148397),
             'Seattle - DARRINGTON - FIR ST (Darrington High School)': (48.2469,
                                                                        -121.6031),
             'Seattle - KENT - JAMES & CENTRAL': (47.386111, -122.230278),
             'Seattle - LAKE FOREST PARK TOWNE CENTER': (47.755, -122.2806),
             'Seattle - MARYSVILLE - 7TH AVE (Marysville Junior High)': (48.054315,
                                                                         -122.171529),
             'Seattle - NORTH BEND - NORTH BEND WAY': (47.49022, -121.77278),
             'Seattle - SEATTLE - BEACON

In [135]:
# with open("target_cities.json", "w") as json_file:
#     json.dump(target_cities, json_file)

In [136]:
import folium

# Create a folium map centered on the first location in the list
map = folium.Map(zoom_start=6)

# Add markers for each location to the map
for city in target_cities["EU"]:
    latitude, longitude = get_city_coordinates(city)
    folium.Marker(location=[latitude, longitude]).add_to(map)

# Save the map to an HTML file
map.save("map_eu.html")

In [137]:
# Create a folium map centered on the first location in the list
map = folium.Map(zoom_start=6)

# Add markers for each location to the map
for city in target_cities["US"]:
    latitude, longitude = get_city_coordinates(city)
    folium.Marker(location=[latitude, longitude]).add_to(map)

# Save the map to an HTML file
map.save("map_usa.html")

In [138]:
# Create a folium map centered on the first location in the list
map = folium.Map(zoom_start=6)

# Add markers for each location to the map
for city in target_cities["Seattle"]:
    latitude, longitude = target_cities["Seattle"][city]
    folium.Marker(location=[latitude, longitude]).add_to(map)

# Save the map to an HTML file
map.save("map_seattle.html")

## ALL target cities on the one map

In [139]:
# Create a folium map centered on the first location in the list
map = folium.Map(location=[42.57, -44.092], zoom_start=3)

for city in target_cities["EU"]:
    latitude, longitude = get_city_coordinates(city)
    folium.Marker(location=[latitude, longitude]).add_to(map)
    
for city in target_cities["US"]:
    latitude, longitude = get_city_coordinates(city)
    folium.Marker(location=[latitude, longitude]).add_to(map)

for city in target_cities["Seattle"]:
    latitude, longitude = target_cities["Seattle"][city]
    folium.Marker(location=[latitude, longitude]).add_to(map)

# Save the map to an HTML file
map.save("map_all_target_cities.html")

In [282]:
# map

In [127]:
# target_cities

In [279]:
def convert_date_to_unix(x):
    """
    Convert datetime to unix time in milliseconds.
    """
    dt_obj = datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
    dt_obj = int(dt_obj.timestamp() * 1000)
    return dt_obj

In [280]:
def get_city_coordinates(city_name: str):
    """
    Takes city name and returns its latitude and longitude (rounded to 2 digits after dot).
    """ 
    # Initialize Nominatim API (for getting lat and long of the city)
    geolocator = Nominatim(user_agent="MyApp")
    city = geolocator.geocode(city_name)

    latitude = round(city.latitude, 2)
    longitude = round(city.longitude, 2)
    
    return latitude, longitude

# Notes

In [76]:
# with open('target_cities.json', "w") as json_file:
#     json.dump(target_cities, json_file)

In [ ]:
# import ast

# string = "(47.1864, -122.4517)"
# t = ast.literal_eval(string)
# t, type(t)

In [ ]:
# seattle_info = seattle_df[["city_name", "latitude", "longitude"]].drop_duplicates()
# for city_name, lat, long in seattle_info.values:
#     target_cities["Seattle"][city_name] = (lat, long)

# [EEA](https://discomap.eea.europa.eu/map/fme/AirQualityExport.htm)
## EEA means European Environmental Agency

In [4]:
def convert_to_daily(df, pollutant: str):
    """
    Returns DataFrame where pollutant column is resampled to days and rounded.
    """
    res_df = df.copy()
    # convert dates in 'time' column
    res_df["date"] = pd.to_datetime(res_df["date"])
    
    # I want data daily, not hourly (mean per each day = 1 datarow per 1 day)
    res_df = res_df.set_index('date')
    res_df = res_df[pollutant].resample('1d').mean().reset_index()
    res_df[pollutant] = res_df[pollutant].fillna(res_df[pollutant].median())
    res_df[pollutant] = res_df[pollutant].apply(lambda x: round(x, 0))
    
    return res_df

In [5]:
def find_fullest_csv(csv_links: list, year: str):
    candidates = [link for link in csv_links if str(year) in link]
    biggest_df = pd.read_csv(candidates[0])
    for link in candidates[1:]:
        _df = pd.read_csv(link)
        if len(biggest_df) < len(_df):
            biggest_df = _df
    return biggest_df

In [6]:
def get_air_quality_from_eea(city_name: str,
                             pollutant: str,
                             start_year: str,
                             end_year: str):
    """
    Takes city name, daterange and returns pandas DataFrame with daily air quality data.
    It parses data by 1-year batches, so please specify years, not dates. (example: "2014", "2022"...)
    
    EEA means European Environmental Agency. So it has data for Europe Union countries ONLY.
    """
    start_of_cell = time.time()
    
    params = {
        'CountryCode': '',
        'CityName': city_name,
        'Pollutant': pollutant.upper(),
        'Year_from': start_year,
        'Year_to': end_year,
        'Station': '',
        'Source': 'All',
        'Samplingpoint': '',
        'Output': 'TEXT',
        'UpdateDate': '',
        'TimeCoverage': 'Year'
    }

    # observations endpoint
    base_url = "https://fme.discomap.eea.europa.eu/fmedatastreaming/AirQualityDownload/AQData_Extract.fmw?"

    response = requests.get(base_url, params=params)

    response.encoding = response.apparent_encoding
    csv_links = response.text.split("\r\n")
    
    res_df = pd.DataFrame()
    target_year = int(start_year)
    
    for year in range(int(start_year), int(end_year) + 1):
        try:
            # find the fullest, the biggest csv file with observations for this particular year
            _df = find_fullest_csv(csv_links, year)
            # append it to res_df
            res_df = pd.concat([res_df, _df])
        except IndexError:
            print(f"!! Missing data for {year} for {city} city.")
            pass
        
    
    pollutant = pollutant.lower()
    if pollutant == "pm2.5":
        pollutant = "pm2_5"
        
    res_df = res_df.rename(columns={
        'DatetimeBegin': 'date',
        'Concentration': pollutant        
    })
    
    # cut timezones info
    res_df['date'] = res_df['date'].apply(lambda x: x[:-6])
    # convert dates in 'time' column
    res_df['date'] = pd.to_datetime(res_df['date'])
    
    res_df = convert_to_daily(res_df, pollutant)
    
    res_df['city_name'] = city_name
    res_df = res_df[['city_name', 'date', pollutant.lower()]]
    
    end_of_cell = time.time()
    
    print(f"Processed {pollutant.upper()} for {city_name} since {start_year} till {end_year}.")
    print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")
    
    return res_df

In [7]:
df_eea_ = get_air_quality_from_eea(
    city_name="Barcelona", pollutant="PM2.5",
    start_year="2013", end_year="2013"
)

df_eea_.head(3)

Processed PM2_5 for Barcelona since 2013 till 2013.
Took 11.16 sec.



,city_name,date,pm2_5
0,Barcelona,2013-01-01,19.0
1,Barcelona,2013-01-02,23.0
2,Barcelona,2013-01-03,28.0


In [9]:
target_cities["EU"]

['Amsterdam',
 'Athina',
 'Berlin',
 'Gdansk',
 'Kraków',
 'London',
 'Madrid',
 'Marseille',
 'Milano',
 'München',
 'Napoli',
 'Paris',
 'Sevilla',
 'Stockholm',
 'Tallinn',
 'Varna',
 'Wien']

In [10]:
start_of_cell = time.time()

pollutant = "PM2.5" # it will become "pm2_5" anyway
start_year = 2013
end_year = 2023

df_eu = pd.DataFrame()

for city in target_cities["EU"]:
    df_ = get_air_quality_from_eea(
        city_name=city, pollutant=pollutant,
        start_year=start_year, end_year=end_year
    )
    df_eu = pd.concat([df_eu, df_])

end_of_cell = time.time()
print("-" * 64)
print(f"Processed {pollutant.upper()} for EU cities since {start_year} till {end_year}.")
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

Processed PM2_5 for Amsterdam since 2013 till 2023.
Took 61.22 sec.

Processed PM2_5 for Athina since 2013 till 2023.
Took 25.29 sec.

Processed PM2_5 for Berlin since 2013 till 2023.
Took 42.02 sec.

Processed PM2_5 for Gdansk since 2013 till 2023.
Took 12.21 sec.

Processed PM2_5 for Kraków since 2013 till 2023.
Took 17.68 sec.

Processed PM2_5 for London since 2013 till 2023.
Took 70.08 sec.

Processed PM2_5 for Madrid since 2013 till 2023.
Took 56.7 sec.

Processed PM2_5 for Marseille since 2013 till 2023.
Took 21.62 sec.

Processed PM2_5 for Milano since 2013 till 2023.
Took 22.92 sec.

Processed PM2_5 for München since 2013 till 2023.
Took 48.32 sec.

Processed PM2_5 for Napoli since 2013 till 2023.
Took 48.52 sec.

Processed PM2_5 for Paris since 2013 till 2023.
Took 54.6 sec.

Processed PM2_5 for Sevilla since 2013 till 2023.
Took 9.27 sec.

Processed PM2_5 for Stockholm since 2013 till 2023.
Took 62.78 sec.

Processed PM2_5 for Tallinn since 2013 till 2023.
Took 5.84 sec.

!! 

In [82]:
df_eu

,city_name,date,pm2_5
0,Amsterdam,2013-01-01,14.0
1,Amsterdam,2013-01-02,8.0
2,Amsterdam,2013-01-03,12.0
3,Amsterdam,2013-01-04,12.0
4,Amsterdam,2013-01-05,14.0
...,...,...,...
3748,Wien,2023-04-07,20.0
3749,Wien,2023-04-08,10.0
3750,Wien,2023-04-09,15.0
3751,Wien,2023-04-10,18.0


In [81]:
df_eu.isna().sum().sum()

0

In [142]:
df_eu.to_csv("data/pm2_5_eu.csv", index=False)

# [USEPA](https://aqs.epa.gov/aqsweb/documents/data_api.html#daily)
## USEPA means United States Environmental Protection Agency
[Manual downloading](https://www.epa.gov/outdoor-air-quality-data/download-daily-data)

In [31]:
city_code_dict = {}
pollutant_dict = {
    'CO': '42101',
    'SO2': '42401',
    'NO2': '42602',
    'O3': '44201',
    'PM10': '81102',
    'PM2.5': '88101'
}

def get_city_code(city_name: str):
    "Encodes city name to be used later for data parsing using USEPA."
    if city_code_dict:
        city_full = [i for i in city_code_dict.keys() if city_name in i][0]
        return city_code_dict[city_full]
    else:
        params = {
            "email": "test@aqs.api",
            "key": "test"
        }
        response = requests.get("https://aqs.epa.gov/data/api/list/cbsas?", params)
        response_json = response.json()
        data = response_json["Data"]
        for item in data:
            city_code_dict[item['value_represented']] = item['code']
        
        return get_city_code(city_name)

In [32]:
def make_date_intervals(start_date, end_date):
    start_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    date_intervals = []
    for year in range(start_dt.year, end_dt.year + 1):
        year_start = datetime.datetime(year, 1, 1)
        year_end = datetime.datetime(year, 12, 31)
        interval_start = max(start_dt, year_start)
        interval_end = min(end_dt, year_end)
        if interval_start < interval_end:
            date_intervals.append((interval_start.strftime('%Y%m%d'), interval_end.strftime('%Y%m%d')))
    return date_intervals


In [44]:
def get_air_quality_from_usepa(city_name: str,
                               pollutant: str,
                               start_date: str,
                               end_date: str):
    """
    Takes city name, daterange and returns pandas DataFrame with daily air quality data.
    
    USEPA means United States Environmental Protection Agency. So it has data for US ONLY.
    """
    start_of_cell = time.time()
        
    res_df = pd.DataFrame()
    
#     # to print 'Success' log only once.
#     was = False
    for start_date_, end_date_ in make_date_intervals(start_date, end_date):
        params = {
            "email": "test@aqs.api",
            "key": "test",
            "param": pollutant_dict[pollutant.upper().replace("_", ".")], # encoded pollutant 
            "bdate": start_date_,
            "edate": end_date_,
            "cbsa": get_city_code(city_name) # Core-based statistical area
        }

        # observations endpoint
        base_url = "https://aqs.epa.gov/data/api/dailyData/byCBSA?" 

        response = requests.get(base_url, params=params)
        response_json = response.json()
        # if not was:
        #     print(response_json["Header"][0]["status"])
        #     was = True
        
        df_ = pd.DataFrame(response_json["Data"])
        
        pollutant = pollutant.lower()
        
        if pollutant == "pm2.5":
            pollutant = "pm2_5"
        df_ = df_.rename(columns={
            'date_local': 'date',
            'arithmetic_mean': pollutant        
        })

        # convert dates in 'date' column
        df_['date'] = pd.to_datetime(df_['date'])
        df_['city_name'] = city_name    
       
        df_ = df_[['city_name', 'date', pollutant]]

        res_df = pd.concat([res_df, df_])
    
    # there are duplicated rows (several records for the same day and station). get rid of it.
    res_df = res_df.groupby(['date', 'city_name'], as_index=False)[pollutant].mean()
    res_df[pollutant] = round(res_df[pollutant], 1)  
    
    end_of_cell = time.time()
    print(f"Processed {pollutant.upper()} for {city_name} since {start_date} till {end_date}.")
    print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")
    
    return res_df

In [72]:
df_usepa_ = get_air_quality_from_usepa(city_name="Albuquerque", pollutant="PM2.5",
                                       start_date="2013-07-01", end_date="2014-01-01")

Processed PM2_5 for Albuquerque since 2013-07-01 till 2014-01-01.
Took 2.16 sec.



In [73]:
df_usepa_

,date,city_name,pm2_5
0,2013-07-01,Albuquerque,4.4
1,2013-07-02,Albuquerque,12.4
2,2013-07-03,Albuquerque,5.6
3,2013-07-04,Albuquerque,12.3
4,2013-07-05,Albuquerque,11.2
...,...,...,...
179,2013-12-27,Albuquerque,8.6
180,2013-12-28,Albuquerque,14.7
181,2013-12-29,Albuquerque,9.3
182,2013-12-30,Albuquerque,9.3


In [75]:
target_cities["US"]

['Albuquerque',
 'Atlanta',
 'Chicago',
 'Columbus',
 'Dallas',
 'Denver',
 'Houston',
 'Los Angeles',
 'New York',
 'Phoenix-Mesa',
 'Salt Lake City',
 'San Francisco',
 'Tampa']

In [77]:
start_of_cell = time.time()

pollutant = "PM2.5"

start_date = "2013-01-01"
end_date = "2023-01-01"

df_us = pd.DataFrame()

for city in target_cities["US"]:
    df_ = get_air_quality_from_usepa(
        city_name=city, pollutant=pollutant,
        start_date=start_date, end_date=end_date
        )
    df_us = pd.concat([df_us, df_])
    
end_of_cell = time.time()
print("-" * 64)
print(f"Processed {pollutant.upper()} for US cities since {start_year} till {end_year}.")
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

Processed PM2_5 for Albuquerque since 2013-01-01 till 2023-01-01.
Took 63.28 sec.

Processed PM2_5 for Atlanta since 2013-01-01 till 2023-01-01.
Took 53.23 sec.

Processed PM2_5 for Chicago since 2013-01-01 till 2023-01-01.
Took 120.86 sec.

Processed PM2_5 for Columbus since 2013-01-01 till 2023-01-01.
Took 42.43 sec.

Processed PM2_5 for Dallas since 2013-01-01 till 2023-01-01.
Took 44.47 sec.

Processed PM2_5 for Denver since 2013-01-01 till 2023-01-01.
Took 72.45 sec.

Processed PM2_5 for Houston since 2013-01-01 till 2023-01-01.
Took 54.07 sec.

Processed PM2_5 for Los Angeles since 2013-01-01 till 2023-01-01.
Took 87.88 sec.

Processed PM2_5 for New York since 2013-01-01 till 2023-01-01.
Took 138.85 sec.

Processed PM2_5 for Phoenix-Mesa since 2013-01-01 till 2023-01-01.
Took 121.5 sec.

Processed PM2_5 for Salt Lake City since 2013-01-01 till 2023-01-01.
Took 109.5 sec.

Processed PM2_5 for San Francisco since 2013-01-01 till 2023-01-01.
Took 100.42 sec.

Processed PM2_5 for Tam

In [78]:
df_us

,date,city_name,pm2_5
0,2013-01-01,Albuquerque,6.8
1,2013-01-02,Albuquerque,8.4
2,2013-01-03,Albuquerque,7.8
3,2013-01-04,Albuquerque,9.2
4,2013-01-05,Albuquerque,12.2
...,...,...,...
3609,2022-12-27,Tampa,6.8
3610,2022-12-28,Tampa,6.9
3611,2022-12-29,Tampa,5.2
3612,2022-12-30,Tampa,5.4


In [79]:
df_us.isna().sum().sum()

0

In [143]:
df_us.to_csv("data/pm2_5_us.csv", index=False)

# Processing special city - `Seattle`.
### We need different stations across the Seattle. 

I downloaded daily `PM2.5` data manually from [here](https://www.epa.gov/outdoor-air-quality-data/download-daily-data)

In [372]:
df_seattle = pd.DataFrame()

for year in range(2013, 2023 + 1):
    df_ = pd.read_csv(f"data/seattle_pm25_{year}.csv")
    df_seattle = pd.concat([df_seattle, df_])

df_seattle = df_seattle.reset_index(drop=True)

df_seattle.shape

(67901, 20)

In [373]:
df_seattle.tail(2)

,Date,Source,Site ID,POC,Daily Mean PM2.5 Concentration,UNITS,DAILY_AQI_VALUE,Site Name,DAILY_OBS_COUNT,PERCENT_COMPLETE,AQS_PARAMETER_CODE,AQS_PARAMETER_DESC,CBSA_CODE,CBSA_NAME,STATE_CODE,STATE,COUNTY_CODE,COUNTY,SITE_LATITUDE,SITE_LONGITUDE
67899,04/02/2023,AirNow,530611007,5,4.8,ug/m3 LC,20,MARYSVILLE - 7TH AVE (Marysville Junior High),1,100.0,88101,PM2.5 - Local Conditions,42660,"Seattle-Tacoma-Bellevue, WA",53,Washington,61,Snohomish,48.054315,-122.171529
67900,04/03/2023,AirNow,530611007,5,4.8,ug/m3 LC,20,MARYSVILLE - 7TH AVE (Marysville Junior High),1,100.0,88101,PM2.5 - Local Conditions,42660,"Seattle-Tacoma-Bellevue, WA",53,Washington,61,Snohomish,48.054315,-122.171529


In [374]:
df_seattle = df_seattle.rename(columns={
    'Daily Mean PM2.5 Concentration': 'pm2_5',
    'Date': 'date',
    'SITE_LATITUDE': 'latitude',
    'SITE_LONGITUDE': 'longitude',
    'Site Name': 'city_name'
})[['city_name', 'date', 'pm2_5', 'latitude', 'longitude']]

In [375]:
df_seattle = df_seattle.drop_duplicates(subset=['date', 'city_name'])

In [376]:
df_seattle.city_name.value_counts()

NORTH BEND - NORTH BEND WAY                                       3705
TACOMA - L STREET                                                 3696
SEATTLE - BEACON HILL                                             3691
MARYSVILLE - 7TH AVE (Marysville Junior High)                     3648
DARRINGTON - FIR ST (Darrington High School)                      3614
SEATTLE - SOUTH PARK #2                                           3577
TACOMA - ALEXANDER AVE                                            3569
KENT - JAMES & CENTRAL                                            3556
SEATTLE - DUWAMISH                                                3439
Seattle-10th & Weller                                             3097
LAKE FOREST PARK TOWNE CENTER                                     2999
PUYALLUP - 128TH ST                                               2700
Tacoma-S 36th St                                                  2574
Bellevue-SE 12th St                                               2172
LYNNWO

In [377]:
df_seattle.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57227 entries, 0 to 67807
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   city_name  56502 non-null  object 
 1   date       57227 non-null  object 
 2   pm2_5      57227 non-null  float64
 3   latitude   57227 non-null  float64
 4   longitude  57227 non-null  float64
dtypes: float64(3), object(2)
memory usage: 2.6+ MB


In [378]:
df_seattle['date'] = pd.to_datetime(df_seattle['date'])

# Group the dataframe by city_name and count the number of rows for each group
counts = df_seattle.groupby('city_name').size()

# Filter the stations that have more than 1000 rows
filtered_counts = counts[counts > 1000]

# Get the city_names of the filtered stations
filtered_sites = filtered_counts.index

# Filter the dataframe to only include rows from the filtered stations
filtered_df = df_seattle[df_seattle['city_name'].isin(filtered_sites)]

# Filter the dataframe to only include rows with a date newer than 2022-08-01
filtered_df = filtered_df[filtered_df['date'] > '2022-08-01']

# Get the city_names of the filtered stations that have at least one row newer than 2022-08-01
newer_sites = filtered_df['city_name'].unique().tolist()

# Create a list of stations that meet both criteria
sites_to_leave = [site for site in filtered_sites if site in newer_sites]

# Print the station list
print(sites_to_leave)

['Bellevue-SE 12th St', 'DARRINGTON - FIR ST (Darrington High School)', 'KENT - JAMES & CENTRAL', 'LAKE FOREST PARK TOWNE CENTER', 'MARYSVILLE - 7TH AVE (Marysville Junior High)', 'NORTH BEND - NORTH BEND WAY', 'SEATTLE - BEACON HILL', 'SEATTLE - DUWAMISH', 'SEATTLE - SOUTH PARK #2', 'Seattle-10th & Weller', 'TACOMA - ALEXANDER AVE', 'TACOMA - L STREET', 'Tacoma-S 36th St', 'Tukwila Allentown', 'Tulalip-Totem Beach Rd']


## Considering data quantity and the freshness of data for each site, I decided to cut off some sites:

In [379]:
df_seattle = df_seattle[df_seattle.city_name.isin(sites_to_leave)].reset_index(drop=True)

In [380]:
df_seattle = df_seattle.dropna()

In [381]:
df_seattle.shape

(46479, 5)

In [382]:
# lets rename these sites so we could later concat this df with other cities data

df_seattle.city_name= df_seattle.city_name.apply(lambda x: "Seattle - " + x)

In [383]:
df_seattle.city_name.value_counts()

Seattle - NORTH BEND - NORTH BEND WAY                      3705
Seattle - TACOMA - L STREET                                3696
Seattle - SEATTLE - BEACON HILL                            3691
Seattle - MARYSVILLE - 7TH AVE (Marysville Junior High)    3648
Seattle - DARRINGTON - FIR ST (Darrington High School)     3614
Seattle - SEATTLE - SOUTH PARK #2                          3577
Seattle - TACOMA - ALEXANDER AVE                           3569
Seattle - KENT - JAMES & CENTRAL                           3556
Seattle - SEATTLE - DUWAMISH                               3439
Seattle - Seattle-10th & Weller                            3097
Seattle - LAKE FOREST PARK TOWNE CENTER                    2999
Seattle - Tacoma-S 36th St                                 2574
Seattle - Bellevue-SE 12th St                              2172
Seattle - Tukwila Allentown                                2074
Seattle - Tulalip-Totem Beach Rd                           1068
Name: city_name, dtype: int64

In [384]:
# df_seattle['latitude'] = df_seattle['latitude'].apply(lambda x: round(x, 5))
# df_seattle['longitude'] = df_seattle['longitude'].apply(lambda x: round(x, 5))

# df_seattle["coordinates"] = tuple(zip(df_seattle["latitude"], df_seattle["longitude"]))
df_seattle = df_seattle.drop(columns=["latitude", "longitude"])

In [385]:
df_seattle.sample(1)

,city_name,date,pm2_5
39055,Seattle - DARRINGTON - FIR ST (Darrington High...,2021-12-12,13.8


In [386]:
df_seattle.to_csv("data/pm2_5_seattle.csv", index=False)

In [387]:
df_seattle = pd.read_csv("data/pm2_5_seattle.csv")

In [388]:
df_seattle.head(3)

,city_name,date,pm2_5
0,Seattle - NORTH BEND - NORTH BEND WAY,2013-01-01,4.7
1,Seattle - NORTH BEND - NORTH BEND WAY,2013-01-02,2.8
2,Seattle - NORTH BEND - NORTH BEND WAY,2013-01-03,3.2


In [389]:
df_seattle.shape

(46479, 3)

# Air Quality [Open Meteo](https://open-meteo.com/en/docs/air-quality-api)

### I will be using this functionality to fill gaps / parse recent data for all my target cities (locations).

#### Note that Open Meteo gives data from `2022-08-04` till `(today + 6 days)` time period.

In [354]:
def get_aqi_data_from_open_meteo(city_name: str,
                                 start_date: str,
                                 end_date: str,
                                 coordinates: list = None,
                                 pollutant: str = "pm2_5"):
    """
    Takes [city name OR coordinates] and returns pandas DataFrame with AQI data.
    
    Examples of arguments:
        ...
        coordinates=(47.755, -122.2806),
        start_date="2023-01-01",
        pollutant="no2"
        ...
    """
    start_of_cell = time.time()
    
    if coordinates:
        latitude, longitude = coordinates
    else:
        latitude, longitude = get_city_coordinates(city_name=city_name)
    
    pollutant = pollutant.lower()
    if pollutant == "pm2.5":
        pollutant = "pm2_5"
    
    # make it work with both "no2" and "nitrogen_dioxide" passed.
    if pollutant == "no2":
        pollutant = "nitrogen_dioxide"
        
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': [pollutant],
        'start_date': start_date,
        'end_date': end_date,
        'timezone': "Europe/London"
    }
    
    # base endpoint
    base_url = "https://air-quality-api.open-meteo.com/v1/air-quality"
        
    response = requests.get(base_url, params=params)

    response_json = response.json()    
    res_df = pd.DataFrame(response_json["hourly"])       
    
    # convert dates
    res_df["time"] = pd.to_datetime(res_df["time"])
    
    # resample to days
    res_df = res_df.groupby(res_df['time'].dt.date).mean(numeric_only=True).reset_index()
    
    res_df[pollutant] = round(res_df[pollutant], 1)
    
    # rename columns
    res_df = res_df.rename(columns={
        "time": "date"
    })
    
    res_df["city_name"] = city_name
    
    # change columns order
    res_df = res_df[
        ['city_name', 'date', pollutant]
    ]
    
    if coordinates:
        res_df["coordinates"] = str(coordinates)
    
#     # create 'unix' columns
#     res_df["unix_time"] = res_df["base_time"].apply(convert_date_to_unix)

    end_of_cell = time.time()
    print(f"Processed {pollutant.upper()} for {city_name} since {start_date} till {end_date}.")
    print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")
    
    return res_df


In [355]:
df_open_meteo_aqi_ = get_aqi_data_from_open_meteo(city_name="Milano",
                                                  start_date="2022-08-04", end_date="2023-01-01")
df_open_meteo_aqi_

Processed PM2_5 for Milano since 2022-08-04 till 2023-01-01.
Took 0.64 sec.



,city_name,date,pm2_5
0,Milano,2022-08-04,9.7
1,Milano,2022-08-05,11.2
2,Milano,2022-08-06,10.5
3,Milano,2022-08-07,7.8
4,Milano,2022-08-08,10.0
...,...,...,...
146,Milano,2022-12-28,75.8
147,Milano,2022-12-29,72.7
148,Milano,2022-12-30,60.1
149,Milano,2022-12-31,48.9


In [359]:
coords_test = (47.60086, -122.14839)

In [360]:
df = get_aqi_data_from_open_meteo(city_name="somewhere near Seattle", coordinates=coords_test,
                                  start_date="2023-01-10", end_date="2023-01-19")


Processed PM2_5 for somewhere near Seattle since 2023-01-10 till 2023-01-19.
Took 0.24 sec.



In [361]:
df

,city_name,date,pm2_5,coordinates
0,somewhere near Seattle,2023-01-10,3.5,"(47.60086, -122.14839)"
1,somewhere near Seattle,2023-01-11,2.9,"(47.60086, -122.14839)"
2,somewhere near Seattle,2023-01-12,2.0,"(47.60086, -122.14839)"
3,somewhere near Seattle,2023-01-13,2.8,"(47.60086, -122.14839)"
4,somewhere near Seattle,2023-01-14,5.2,"(47.60086, -122.14839)"
5,somewhere near Seattle,2023-01-15,4.6,"(47.60086, -122.14839)"
6,somewhere near Seattle,2023-01-16,3.2,"(47.60086, -122.14839)"
7,somewhere near Seattle,2023-01-17,5.7,"(47.60086, -122.14839)"
8,somewhere near Seattle,2023-01-18,3.7,"(47.60086, -122.14839)"
9,somewhere near Seattle,2023-01-19,6.3,"(47.60086, -122.14839)"


# Filling the gap in Air Quality data (PM2.5)

In [235]:
last_dates_eu = df_eu[["date", "city_name"]].groupby("city_name").max()
last_dates_eu.date = last_dates_eu.date.astype(str)
last_dates_eu = last_dates_eu.to_dict()["date"]

last_dates_us = df_us[["date", "city_name"]].groupby("city_name").max()
last_dates_us.date = last_dates_us.date.astype(str)
last_dates_us = last_dates_us.to_dict()["date"]

last_dates_seattle = df_seattle[["date", "city_name"]].groupby("city_name").max()
last_dates_seattle.date = last_dates_seattle.date.astype(str)
last_dates_seattle = last_dates_seattle.to_dict()["date"]

In [369]:
target_cities["Seattle"]['Seattle - Bellevue-SE 12th St']

(47.6008630009392, -122.148397)

In [344]:
last_dates_eu

{'Amsterdam': '2023-04-11',
 'Athina': '2023-04-11',
 'Berlin': '2023-04-11',
 'Gdansk': '2023-04-11',
 'Kraków': '2023-04-11',
 'London': '2023-04-11',
 'Madrid': '2023-04-11',
 'Marseille': '2023-04-11',
 'Milano': '2023-04-11',
 'München': '2023-04-11',
 'Napoli': '2023-04-11',
 'Paris': '2023-04-11',
 'Sevilla': '2023-04-11',
 'Stockholm': '2023-04-11',
 'Tallinn': '2023-04-11',
 'Varna': '2022-08-02',
 'Wien': '2023-04-11'}

In [153]:
date_today = datetime.datetime.now().strftime("%Y-%m-%d")
date_today

'2023-04-13'

In [311]:
start_of_cell = time.time()

df_eu_update = pd.DataFrame()
for city_name in last_dates_eu: 
    df_ = get_aqi_data_from_open_meteo(city_name=city_name,
                                       start_date=last_dates_eu[city_name],
                                       end_date=date_today)
    df_eu_update = pd.concat([df_eu_update, df_]).reset_index(drop=True)
    
    
end_of_cell = time.time()
print("-" * 64)
print(f"Parsed new PM2.5 data for EU cities up to {date_today}.")
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

Processed PM2_5 for Amsterdam since 2023-04-11 till 2023-04-13.
Took 0.54 sec.

Processed PM2_5 for Athina since 2023-04-11 till 2023-04-13.
Took 0.85 sec.

Processed PM2_5 for Berlin since 2023-04-11 till 2023-04-13.
Took 0.49 sec.

Processed PM2_5 for Gdansk since 2023-04-11 till 2023-04-13.
Took 0.51 sec.

Processed PM2_5 for Kraków since 2023-04-11 till 2023-04-13.
Took 0.55 sec.

Processed PM2_5 for London since 2023-04-11 till 2023-04-13.
Took 0.52 sec.

Processed PM2_5 for Madrid since 2023-04-11 till 2023-04-13.
Took 0.49 sec.

Processed PM2_5 for Marseille since 2023-04-11 till 2023-04-13.
Took 0.5 sec.

Processed PM2_5 for Milano since 2023-04-11 till 2023-04-13.
Took 0.54 sec.

Processed PM2_5 for München since 2023-04-11 till 2023-04-13.
Took 0.47 sec.

Processed PM2_5 for Napoli since 2023-04-11 till 2023-04-13.
Took 0.54 sec.

Processed PM2_5 for Paris since 2023-04-11 till 2023-04-13.
Took 0.57 sec.

Processed PM2_5 for Sevilla since 2023-04-11 till 2023-04-13.
Took 0.51

In [308]:
df_eu_update

,city_name,date,pm2_5
0,Amsterdam,2023-04-11,7.4
1,Amsterdam,2023-04-12,5.5
2,Amsterdam,2023-04-13,5.5
3,Athina,2023-04-11,10.0
4,Athina,2023-04-12,14.4
...,...,...,...
298,Varna,2023-04-12,6.4
299,Varna,2023-04-13,8.5
300,Wien,2023-04-11,16.8
301,Wien,2023-04-12,10.9


In [310]:
start_of_cell = time.time()

df_us_update = pd.DataFrame()
for city_name in last_dates_us: 
    df_ = get_aqi_data_from_open_meteo(city_name=city_name,
                                       start_date=last_dates_us[city_name],
                                       end_date=date_today)
    df_us_update = pd.concat([df_us_update, df_]).reset_index(drop=True)
    
    
end_of_cell = time.time()
print("-" * 64)
print(f"Parsed new PM2.5 data for US cities up to {date_today}.")
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

Processed PM2_5 for Albuquerque since 2022-12-31 till 2023-04-13.
Took 0.57 sec.

Processed PM2_5 for Atlanta since 2022-12-31 till 2023-04-13.
Took 0.51 sec.

Processed PM2_5 for Chicago since 2022-12-31 till 2023-04-13.
Took 0.54 sec.

Processed PM2_5 for Columbus since 2022-12-31 till 2023-04-13.
Took 0.53 sec.

Processed PM2_5 for Dallas since 2022-12-31 till 2023-04-13.
Took 0.54 sec.

Processed PM2_5 for Denver since 2022-12-31 till 2023-04-13.
Took 0.53 sec.

Processed PM2_5 for Houston since 2022-12-31 till 2023-04-13.
Took 0.56 sec.

Processed PM2_5 for Los Angeles since 2022-12-31 till 2023-04-13.
Took 0.91 sec.

Processed PM2_5 for New York since 2022-12-31 till 2023-04-13.
Took 0.54 sec.

Processed PM2_5 for Phoenix-Mesa since 2022-12-31 till 2023-04-13.
Took 0.57 sec.

Processed PM2_5 for Salt Lake City since 2022-12-31 till 2023-04-13.
Took 0.54 sec.

Processed PM2_5 for San Francisco since 2022-12-31 till 2023-04-13.
Took 1.97 sec.

Processed PM2_5 for Tampa since 2022-1

In [313]:
import ast

string = "(47.1864, -122.4517)"
t = ast.literal_eval(string)
t, type(t)

((47.1864, -122.4517), tuple)

In [391]:
last_dates_seattle['Seattle - Bellevue-SE 12th St']

'2023-04-03'

In [393]:
start_of_cell = time.time()

df_seattle_update = pd.DataFrame()
for city_name in last_dates_seattle:
    coordinates = target_cities["Seattle"][city_name]
    df_ = get_aqi_data_from_open_meteo(city_name=city_name,
                                       coordinates=coordinates,
                                       start_date=last_dates_seattle[city_name],
                                       end_date=date_today)
    df_seattle_update = pd.concat([df_seattle_update, df_]).reset_index(drop=True)
    

end_of_cell = time.time()
print("-" * 64)
print(f"Parsed new PM2.5 data for Seattle and surrounding areas up to {date_today}.")
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

Processed PM2_5 for Seattle - Bellevue-SE 12th St since 2023-04-03 till 2023-04-13.
Took 0.22 sec.

Processed PM2_5 for Seattle - DARRINGTON - FIR ST (Darrington High School) since 2023-04-03 till 2023-04-13.
Took 0.29 sec.

Processed PM2_5 for Seattle - KENT - JAMES & CENTRAL since 2023-04-03 till 2023-04-13.
Took 0.29 sec.

Processed PM2_5 for Seattle - LAKE FOREST PARK TOWNE CENTER since 2023-04-03 till 2023-04-13.
Took 0.23 sec.

Processed PM2_5 for Seattle - MARYSVILLE - 7TH AVE (Marysville Junior High) since 2023-04-03 till 2023-04-13.
Took 0.26 sec.

Processed PM2_5 for Seattle - NORTH BEND - NORTH BEND WAY since 2023-04-03 till 2023-04-13.
Took 0.22 sec.

Processed PM2_5 for Seattle - SEATTLE - BEACON HILL since 2023-04-03 till 2023-04-13.
Took 0.27 sec.

Processed PM2_5 for Seattle - SEATTLE - DUWAMISH since 2023-04-03 till 2023-04-13.
Took 0.22 sec.

Processed PM2_5 for Seattle - SEATTLE - SOUTH PARK #2 since 2023-04-03 till 2023-04-13.
Took 0.25 sec.

Processed PM2_5 for Sea

In [394]:
df_seattle_update

,city_name,date,pm2_5,coordinates
0,Seattle - Bellevue-SE 12th St,2023-04-03,6.4,"(47.6008630009392, -122.148397)"
1,Seattle - Bellevue-SE 12th St,2023-04-04,9.9,"(47.6008630009392, -122.148397)"
2,Seattle - Bellevue-SE 12th St,2023-04-05,12.6,"(47.6008630009392, -122.148397)"
3,Seattle - Bellevue-SE 12th St,2023-04-06,12.2,"(47.6008630009392, -122.148397)"
4,Seattle - Bellevue-SE 12th St,2023-04-07,4.7,"(47.6008630009392, -122.148397)"
...,...,...,...,...
161,Seattle - Tulalip-Totem Beach Rd,2023-04-09,9.0,"(48.065339, -122.285194)"
162,Seattle - Tulalip-Totem Beach Rd,2023-04-10,6.3,"(48.065339, -122.285194)"
163,Seattle - Tulalip-Totem Beach Rd,2023-04-11,3.9,"(48.065339, -122.285194)"
164,Seattle - Tulalip-Totem Beach Rd,2023-04-12,9.1,"(48.065339, -122.285194)"


# Weather data from [open meteo](https://open-meteo.com/en/docs)

- Maximum Temperature (2 m)
- Minimum Temperature (2 m)
- Precipitation Sum
- Rain Sum
- Snowfall Sum
- Precipitation Hours
- Maximum Wind Speed (10 m)
- Maximum Wind Gusts (10 m)
- Dominant Wind Direction (10 m)


In [6]:
def get_weather_data_from_open_meteo(city_name: str,
                                     start_date: str,
                                     end_date: str,
                                     coordinates: list = None,
                                     forecast: bool = False):
    """
    Takes [city name OR coordinates] and returns pandas DataFrame with weather data.
    
    Examples of arguments:
        coordinates=(47.755, -122.2806), start_date="2023-01-01"
    """
    
    if coordinates:
        latitude, longitude = coordinates
    else:
        latitude, longitude = get_city_coordinates(city_name=city_name)
    
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'daily': ["temperature_2m_max", "temperature_2m_min",
                  "precipitation_sum", "rain_sum", "snowfall_sum",
                  "precipitation_hours", "windspeed_10m_max",
                  "windgusts_10m_max", "winddirection_10m_dominant"],
        'start_date': start_date,
        'end_date': end_date,
        'timezone': "Europe/London"
    }
    
    if forecast:
        # historical forecast endpoint
        base_url = 'https://api.open-meteo.com/v1/forecast' 
    else:
        # historical observations endpoint
        base_url = 'https://archive-api.open-meteo.com/v1/archive' 
        
    response = requests.get(base_url, params=params)

    response_json = response.json()    
    res_df = pd.DataFrame(response_json["daily"])
    
    res_df["city_name"] = city_name
    
    # rename columns
    res_df = res_df.rename(columns={
        "time": "date",
        "temperature_2m_max": "temperature_max",
        "temperature_2m_min": "temperature_min",
        "windspeed_10m_max": "wind_speed_max",
        "winddirection_10m_dominant": "wind_direction_dominant",
        "windgusts_10m_max": "wind_gusts_max"
    })
    
    # change columns order
    res_df = res_df[
        ['city_name', 'date', 'temperature_max', 'temperature_min',
         'precipitation_sum', 'rain_sum', 'snowfall_sum',
         'precipitation_hours', 'wind_speed_max',
         'wind_gusts_max', 'wind_direction_dominant']
    ]
    
    # convert dates in 'date' column
    res_df["date"] = pd.to_datetime(res_df["date"])
    
#     # create 'unix' columns
#     res_df["unix_time"] = res_df["base_time"].apply(convert_date_to_unix)
    
    return res_df

In [80]:
get_weather_data_from_open_meteo(city_name="some site in Seattle", 
                                 coordinates=(47.755, -122.2806), start_date="2023-01-01", end_date="2023-01-01")

,city_name,date,temperature_max,temperature_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_max,wind_gusts_max,wind_direction_dominant
0,some site in Seattle,2023-01-01,8.1,3.9,2.2,2.2,0.0,10.0,7.8,14.8,140


In [81]:
get_weather_data_from_open_meteo(city_name="Seattle", start_date="2023-01-01", end_date="2023-01-01")

,city_name,date,temperature_max,temperature_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_max,wind_gusts_max,wind_direction_dominant
0,Seattle,2023-01-01,8.6,2.7,1.8,1.8,0.0,8.0,7.1,23.0,155


In [8]:
get_weather_data_from_open_meteo(city_name="Seattle", start_date="2023-01-01", end_date="2023-01-01", forecast=1)

,city_name,date,temperature_max,temperature_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_max,wind_gusts_max,wind_direction_dominant
0,Seattle,2023-01-01,8.6,4.5,0.0,0.0,0.0,0.0,8.1,15.1,99


In [9]:
seattle_df = pd.read_csv("data/processed_seattle_pm25_2013_2022.csv")

In [10]:
row = seattle_df.sample(1)
row

,site_name,date,pm25,latitude,longitude
45811,Seattle - TACOMA - ALEXANDER AVE,2021-10-24,3.6,47.2656,-122.3858


In [11]:
city_name = row.city_name.values[0]
coordinates = list(row[['latitude', 'longitude']].values[0])

In [12]:
city_name, coordinates

('Seattle - TACOMA - ALEXANDER AVE', [47.2656, -122.3858])

In [13]:
get_weather_data_from_open_meteo(city_name=city_name, coordinates=coordinates, start_date="2023-01-01", end_date="2023-01-01")

,city_name,date,temperature_max,temperature_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_max,wind_gusts_max,wind_direction_dominant
0,Seattle - TACOMA - ALEXANDER AVE,2023-01-01,8.1,3.4,0.1,0.1,0.0,1.0,10.1,25.6,182


In [17]:
get_weather_data_from_open_meteo(city_name="Colorado Springs", start_date=date_today, end_date=date_today, forecast=True)

,city_name,date,temperature_max,temperature_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_max,wind_gusts_max,wind_direction_dominant
0,Colorado Springs,2023-04-06,10.1,-4.6,3.3,4.35,0.28,5.0,19.5,25.9,154


# TODO: Data engineering - MOVE IT TO THE NEXT NOTEBOOK

In [ ]:
# Data engineering

def moving_average(df, window=7):
    df[f'mean_{window}_days'] = df.groupby('station_id')['users_count'] \
                                    .rolling(window=window).mean().reset_index(0,drop=True).shift(1)
    return df

# def moving_average(df, window=7):
#     df[f'mean_{window}_days'] = df["users_count"].rolling(window=window).mean()
#     return df


def moving_std(df, window):
    df[f'std_{window}_days'] = df.groupby('station_id')['users_count'] \
                                    .rolling(window=window).std().reset_index(0,drop=True).shift(1)
    return df


def exponential_moving_average(df, window):
    df[f'exp_mean_{window}_days'] = df.groupby('station_id')['users_count'].ewm(span=window) \
                                        .mean().reset_index(0,drop=True).shift(1)
    return df


def exponential_moving_std(df, window):
    df[f'exp_std_{window}_days'] = df.groupby('station_id')['users_count'].ewm(span=window) \
                                        .std().reset_index(0,drop=True).shift(1)
    return df


def engineer_citibike_features(df):
    df_res = df.copy()
    # there are duplicated rows (several records for the same day and station). get rid of it.
    df_res = df_res.groupby(['date', 'station_id'], as_index=False)['users_count'].sum()

    df_res['prev_users_count'] = df_res.groupby('station_id')['users_count'].shift(+1)
    df_res = df_res.dropna()
    df_res = moving_average(df_res, 7)
    df_res = moving_average(df_res, 14)


    for i in [7, 14]:
        for func in [moving_std, exponential_moving_average,
                     exponential_moving_std
                     ]:
            df_res = func(df_res, i)
    df_res = df_res.reset_index(drop=True)
    return df_res.sort_values(by=["date", "station_id"]).dropna()